## Analysing performance of a simple SVM classifier

In [1]:
import sys

sys.path.insert(0, "..")

from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from HAR.transformers import CSIMinMaxScaler
from HAR.io import load_dataset

### [1] Train and test SVM on data captured on some subset of days

In [2]:
X, y, _, classnames, dim = load_dataset("../dataset/rCSI-d1d2.mat")
X = X.reshape(X.shape[0], *dim)
X = CSIMinMaxScaler().fit_transform(X)
X = X.reshape(X.shape[0], dim[0] * dim[1])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train an SVM classifier on the training data
clf = svm.SVC(kernel="rbf", decision_function_shape="ovo")
clf.fit(X_train, y_train)

# Predict the class labels for the test data
y_pred = clf.predict(X_test)

print(f"\n> Test Accuracy: {accuracy_score(y_test, y_pred)*100:.4f}%")
print("\n> Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))
print("\n> Classification Report :")
print(classification_report(y_test, y_pred, target_names=classnames))


> Test Accuracy: 97.6667%

> Confusion Matrix: 
[[100   0   0]
 [  1  95   4]
 [  0   2  98]]

> Classification Report :
              precision    recall  f1-score   support

        Idle       0.99      1.00      1.00       100
        Walk       0.98      0.95      0.96       100
        Jump       0.96      0.98      0.97       100

    accuracy                           0.98       300
   macro avg       0.98      0.98      0.98       300
weighted avg       0.98      0.98      0.98       300



### [2] Test SVM on data captured on a different day, not part of previous train-test set

In [3]:
import warnings

warnings.filterwarnings("ignore")

X, y, _, classnames, dim = load_dataset("../dataset/rCSI-d3.mat")
X = X.reshape(X.shape[0], *dim)
X = CSIMinMaxScaler().fit_transform(X)
X = X.reshape(X.shape[0], dim[0] * dim[1])

y_pred = clf.predict(X)

print(f"\n> Test Accuracy: {accuracy_score(y, y_pred)*100:.4f}%")
print("\n> Confusion Matrix: ")
print(confusion_matrix(y, y_pred))
print("\n> Classification Report :")
print(classification_report(y, y_pred, target_names=classnames))


> Test Accuracy: 36.0784%

> Confusion Matrix: 
[[ 14  94  62]
 [  0 170   0]
 [  0 170   0]]

> Classification Report :
              precision    recall  f1-score   support

        Idle       1.00      0.08      0.15       170
        Walk       0.39      1.00      0.56       170
        Jump       0.00      0.00      0.00       170

    accuracy                           0.36       510
   macro avg       0.46      0.36      0.24       510
weighted avg       0.46      0.36      0.24       510



**Observation**: We observe a sharp drop in prediction accuracy as concluded at the end of t-SNE analysis.